In [ ]:
import networkx as nx
import numpy as np
import pickle as p
from os import path
from matplotlib import pyplot as plt
%matplotlib inline
import torch
data_loc = './data/BlogCatalog3/BlogCatalog-dataset/data/'
import pickle

In [ ]:
import torch
import numpy as np
import random

def gen_random_walk_tensor(graph, node, length, num_walks):
    walk = torch.zeros((num_walks, length), dtype=int)
    walk[:, 0] = node
    j = 0
    while j < num_walks:
        current_node = node
        step = 1
        while step < length:
            neighbors = list(graph.neighbors(current_node))
            current_node = random.choice(neighbors)
            walk[j, step] = current_node
            step += 1
        j+=1
    return walk

def gen_batch_random_walk(graph, initial_nodes, length, num_walks):
    n_nodes = initial_nodes.shape[0]
    walk = torch.zeros((num_walks*n_nodes, length), dtype=int)
    for i, n in enumerate(initial_nodes):
        n = n.item()
        walk[num_walks*i:num_walks*(i+1)] = gen_random_walk_tensor(graph, n, length, num_walks)
    return walk

def generate_windows(random_walk, window_size):
    num_walks, walk_length = random_walk.shape
    # number of windows: e.g. length 5, window size 3 -> 3 windows ([0, 1, 2], [1, 2, 3], [2, 3, 4])
    num_windows = walk_length + 1 - window_size
    windows = torch.zeros((num_walks*num_windows, window_size), dtype=int)
    for j in range(num_windows):
        windows[num_walks*j:num_walks*(j+1)] = random_walk[:, j:j+window_size]
    return windows

def get_windows_dotproduct(windows, embedding):
    embedding_size = embedding.shape[1]
    # get the embedding of the initial node repeated num_windows times
    first_emb = embedding[windows[:, 0]]
    first_emb = first_emb.view(windows.shape[0], 1, embedding_size)
    # get the embedding of the remaining nodes in each window
    others_emb = embedding[windows[:, 1:]]
    others_emb = others_emb.view(windows.shape[0], -1, embedding_size)
    # result has same shape as others
    # Each element is the dot product between the corresponding node embedding
    # and the embedding of the first node of that walk
    # that is, result_{i, j} for random walk i and element j is v_{W_{i, 0}} dot v_{W_{i, j}}
    result = (first_emb*others_emb).sum(dim=-1)
    return result

def gen_negative_samples(amount, length, initial_node, number_of_nodes):
    negative_samples = torch.zeros((amount, length), dtype=int)
    negative_samples[:, 0] = initial_node
    negative_samples[:, 1:] = torch.randint(number_of_nodes, (amount, length-1))
    return negative_samples

def gen_batch_negative_samples(amount, length, initial_nodes, number_of_nodes, distribution=None):
    negative_samples = torch.zeros((amount*initial_nodes.shape[0], length), dtype=int)
    negative_samples[:, 0] = initial_nodes.repeat(amount, 1).t().contiguous().view(-1)
    if distribution is None:
        negative_samples[:, 1:] = torch.randint(number_of_nodes, (amount*initial_nodes.shape[0], length-1))
    else:
        M = amount*initial_nodes.shape[0]
        N = length-1
        negative_samples[:, 1:] = torch.multinomial(distribution, M*N, replacement=True).view(M, N)
    return negative_samples

def gen_biaised_random_walk_tensor(graph, start_node, walk_length, num_walks, p, q , neighbors_dict):
    walks = torch.zeros((num_walks, walk_length), dtype=int)
    walks[:, 0] = start_node

    for walk_index in range(num_walks):
        current_node = start_node
        for step in range(walk_length):
            walks[walk_index, step] = current_node
            if step > 0:
                prev_node = int(walks[walk_index, step - 1])
                current_node = get_next_node(graph,prev_node,current_node,p,q)
            else:
                current_node = np.random.choice(list(graph.neighbors(current_node)))
            
    
    return walks

def get_next_node(graph,t,v, p, q):
    v_neighbors = set(graph.neighbors(v))
    t_neighbors = set(graph.neighbors(t))
    t_set = set([t])

    vt_neighbors = v_neighbors & t_neighbors
    only_v_neighbors = v_neighbors - t_neighbors - t_set

    allsets = [vt_neighbors,only_v_neighbors,t_set]

    vt_weights = 1 * len(vt_neighbors)
    only_v_weights = 1/q * len(only_v_neighbors)
    t_weight = 1/p

    prob_vector = np.array((vt_weights,only_v_weights,t_weight))
    prob_vector = prob_vector / np.sum(prob_vector)

    chosen_set = np.random.choice(allsets,p=prob_vector)
    next_node = np.random.choice(list(chosen_set))
    return next_node

def gen_batch_biaised_random_walk(graph, initial_nodes, length, num_walks, p, q, neighbors_dict):
    n_nodes = initial_nodes.shape[0]
    walk = torch.zeros((num_walks*n_nodes, length), dtype=int)
    for i, n in enumerate(initial_nodes):
        n = n.item()
        walk[num_walks*i:num_walks*(i+1)] = gen_biaised_random_walk_tensor(graph, n, length, num_walks, p, q,neighbors_dict)
    return walk

In [ ]:
import psutil

In [ ]:
from tqdm import tqdm
import torch
import numpy as np
import random

eps = 1e-15

def generate_batches(array, batch_size):
    """Yield successive batches of size `batch_size` from `array`."""
    for i in range(0, len(array), batch_size):
        yield array[i:i + batch_size]

def deepWalk(graph, walks_per_vertex, walk_length, window_size, embedding_size, num_neg, lr, epochs, batch_size, distribution=None):
    number_of_nodes = graph.number_of_nodes()
    
    embedding = (torch.randn(size=(number_of_nodes, embedding_size))).detach()
    embedding.requires_grad = True
    optimizer = torch.optim.Adam([embedding], lr=lr)
    loss_history = {'pos': [], 'neg': [], 'total': []}
    nodes = torch.tensor(list(graph.nodes), dtype=int)

    for _ in range(epochs):
        random_ixs = torch.randperm(nodes.shape[0])
        nodes = nodes[random_ixs]
        node_loader = generate_batches(nodes, batch_size)
        n_batches = int(number_of_nodes / batch_size)
        for n in tqdm(node_loader, total=n_batches):

            random_walk = gen_batch_random_walk(graph, n, walk_length, walks_per_vertex)
            # Positive Sampling
            # each row of windows is one window, we have B = walks_per_vertex*num_windows windows
            windows = generate_windows(random_walk, window_size)
            batch_dotproduct = get_windows_dotproduct(windows, embedding)
            # takes the sigmoid of the dot product to get probability, then
            # takes the loglik and average through all elements
            pos_loss = -torch.log(torch.sigmoid(batch_dotproduct)+eps).mean()
            # Negative Sampling
            negative_samples = gen_batch_negative_samples(
                amount=num_neg*walks_per_vertex, 
                length=walk_length, 
                initial_nodes=n, 
                number_of_nodes=number_of_nodes,
                distribution=distribution
            )
            windows = generate_windows(negative_samples, window_size)
            batch_dotproduct = get_windows_dotproduct(windows, embedding)
            neg_loss = -torch.log(1-torch.sigmoid(batch_dotproduct)+eps).mean()

            loss = pos_loss + neg_loss
            # Optimization
            loss.backward()
            loss_history['total'].append(loss.detach().numpy())
            loss_history['pos'].append(pos_loss.detach().numpy())
            loss_history['neg'].append(neg_loss.detach().numpy())
            optimizer.step()
            optimizer.zero_grad()  

    return embedding, loss_history

def node2vec(graph, walks_per_vertex, walk_length, window_size, embedding_size, num_neg, lr, epochs, batch_size, p = 5, q = 5):
    number_of_nodes = graph.number_of_nodes()
    
    embedding = (torch.randn(size=(number_of_nodes, embedding_size)) ).detach()
    embedding.requires_grad = True
    optimizer = torch.optim.Adam([embedding], lr=lr)
    loss_history = {'pos': [], 'neg': [], 'total': []}
    neighbors_dict = {node: list(graph.neighbors(node)) for node in graph.nodes}
    nodes = torch.tensor(list(graph.nodes), dtype=int)

    for _ in range(epochs):
        random.shuffle(nodes)
        node_loader = generate_batches(nodes, batch_size)
        n_batches = int(number_of_nodes / batch_size)
        for n in tqdm(node_loader, total=n_batches):
            random_walk = gen_batch_biaised_random_walk(graph, n, walk_length, walks_per_vertex, p, q, neighbors_dict)
            num_windows = walk_length + 1 - window_size

            # Positive Sampling
            # each row of windows is one window, we have B = walks_per_vertex*num_windows windows
            windows = generate_windows(random_walk, window_size)
            batch_dotproduct = get_windows_dotproduct(windows, embedding)
            # takes the sigmoid of the dot product to get probability, then
            # takes the loglik and average through all elements
            pos_loss = -torch.log(torch.sigmoid(batch_dotproduct)+eps).mean()
            # Negative Sampling
            negative_samples = gen_batch_negative_samples(
                amount=num_neg*walks_per_vertex, 
                length=walk_length, 
                initial_nodes=n, 
                number_of_nodes=number_of_nodes
            )
            windows = generate_windows(negative_samples, window_size)
            batch_dotproduct = get_windows_dotproduct(windows, embedding)
            neg_loss = -torch.log(1-torch.sigmoid(batch_dotproduct)+eps).mean()

            loss = pos_loss + neg_loss
            # Optimization
            loss.backward()
            loss_history['total'].append(loss.detach().numpy())
            loss_history['pos'].append(pos_loss.detach().numpy())
            loss_history['neg'].append(neg_loss.detach().numpy())
            optimizer.step()
            optimizer.zero_grad()
    return embedding, loss_history


## Load Data

In [ ]:
def load_data():
    iid = {}
    idx = 0
    edgelist = []

    # Read edges pairs
    with open(data_loc+'edges.csv', 'r') as f:
        for line in f.readlines():
            i, j = line.strip().split(',')  # csv
            if i not in iid:
                iid[i] = idx; idx += 1
            if j not in iid:
                iid[j] = idx; idx += 1
            edgelist.append((iid[i], iid[j]))

    # Create an nx undirected network
    bc = nx.Graph(edgelist)

    print("Number of nodes: ", len(bc))
    print("Number of edges: ", bc.size())

    # Read labels
    labels = np.zeros((len(bc)), dtype=int)
    # Read (node_id, label) file
    with open(data_loc+'group-edges.csv', 'r') as f:
        for line in f.readlines():
            node, group = line.strip().split(',') 
            labels[iid[node]] = int(group)-1  

    bc_dataset = {'graph': bc, 'labels': labels}
    return bc_dataset

bc_dataset = load_data()

In [ ]:
import logging
logging.basicConfig(filename='./deepWalk_bayesian_opt.log', filemode='w',level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logging.info("Starting optimization")


In [ ]:
from skopt.space import Real, Categorical, Integer
from skopt import gp_minimize
from skopt.utils import use_named_args
from IPython.display import clear_output

walks_per_vertex = 1
embedding_size = 128
window_size = 10
walk_length = 80
epochs = 10
batch_size = 64

dim_num_neg = Integer(1,10,name="num_neg")
dim_lr = Real(
    low=1e-3, high=1e-1, prior='log-uniform', name='lr',
)


# the hyperparameter space grid
param_grid = [dim_num_neg,dim_lr]


@use_named_args(param_grid)
def objective(num_neg,lr):    
    
    clear_output(wait=True)
    print("Current Training")
    print(f"    walks_per_vertex: {walks_per_vertex}")
    print(f"    walk_length: {walk_length}")
    print(f"    window_size: {window_size}")
    print(f"    embedding_size: {embedding_size}")
    print(f"    num_neg: {num_neg}")
    print(f"    lr: {lr}")
    print(f"    epochs: {epochs}")
    print(f"    batch_size: {batch_size}")

    logging.info("Starting optimization with following parameters:")
    logging.info(f"     walks_per_vertex: {walks_per_vertex}")
    logging.info(f"     walk_length: {walk_length}")
    logging.info(f"     window_size: {window_size}")
    logging.info(f"     embedding_size: {embedding_size}")
    logging.info(f"     num_neg: {num_neg}")
    logging.info(f"     lr: {lr}")
    logging.info(f"     epochs: {epochs}")
    logging.info(f"     batch_size: {batch_size}")





    embedding, loss_history = deepWalk(graph=bc_dataset['graph'],
                                walks_per_vertex=walks_per_vertex,
                                walk_length=walk_length,
                                window_size=window_size,
                                embedding_size=embedding_size,
                                num_neg=num_neg,
                                lr=lr,
                                epochs=epochs,
                                batch_size=batch_size)
    logging.info(f"Final loss: {loss_history['total'][-1]}")
    return float(loss_history['total'][-1])


default_parameters = [3,1e-2]

In [ ]:
gp_ = gp_minimize(
    objective, # the objective function to minimize
    param_grid, # the hyperparameter space
    x0=default_parameters, # the initial parameters to test
    acq_func='EI', # the acquisition function
    n_calls=20, # the number of subsequent evaluations of f(x)
    random_state=0,
)

In [ ]:
"Best score=%.4f" % gp_.fun

In [ ]:
gp_.x

In [ ]:
with open('./output/Deepwalk_gp.pickle', 'wb') as f:
    pickle.dump(gp_, f)

In [ ]:
num_neg,lr = gp_.x

embedding, loss_history = deepWalk(graph=bc_dataset['graph'],
                                walks_per_vertex=walks_per_vertex,
                                walk_length=walk_length,
                                window_size=window_size,
                                embedding_size=embedding_size,
                                num_neg=num_neg,
                                lr=lr,
                                epochs=epochs,
                                batch_size=batch_size)

In [ ]:
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression

X = embedding.detach().numpy()
y = bc_dataset['labels']

clf = LogisticRegression(random_state=0,max_iter=1000).fit(X, y)
y_hat = clf.predict(X)
f1_score(y, y_hat, average='macro')